In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import pandas as pd
import itertools as itt
hv.notebook_extension('bokeh', width=100)

In [ ]:
mappings_meta_fill = pd.read_pickle("./data/mappings_meta_fill.pkl")

In [ ]:
overlap_list = []
map_dict = {'1': 'A1', '2': 'B1', '3': 'BS', '4': 'A2', '5': 'B2', '6': 'C'}
for cur_anm, cur_grp in mappings_meta_fill.groupby(mappings_meta_fill['meta', 'animal']):
    cur_ss = cur_grp['session'].dropna(axis='columns', how='all').columns
    T = cur_grp['session'].dropna(axis='rows', how='all').shape[0]
    for cur_map in itt.combinations(cur_ss, 2):
        nint = cur_grp['session'][list(cur_map)].dropna(axis='rows', how='any').shape[0]
        nuni = cur_grp['session'][list(cur_map)].dropna(axis='rows', how='all').shape[0]
        nA = cur_grp['session'][cur_map[0]].dropna().size
        nB = cur_grp['session'][cur_map[1]].dropna().size
        cur_map = tuple([map_dict[m] for m in cur_map])
        cur_ovlp = pd.Series([cur_anm, cur_map, nint/nuni, nint/nA, nint/nB, (nint*T)/(nA*nB)], index=['animal', 'session', 'overlap', 'overlap-onA', 'overlap-onB', 'overlap-prod'])
        overlap_list.append(cur_ovlp)
overlaps = pd.concat(overlap_list, axis=1, ignore_index=True).T
group_dict = dict(MS101='negative', MS104='negative', NS20='negative', NS22='negative', MS102='neutral', MS103='neutral', NS24='neutral')
overlaps['group'] = overlaps['animal'].apply(lambda anm: group_dict[anm])

In [ ]:
overlaps = overlaps.melt(id_vars=['animal', 'session', 'group'], var_name='overlap-type', value_name='overlap-value')
overlaps['overlap-value'] = overlaps['overlap-value'].astype(float)

In [ ]:
%%opts BoxWhisker [width=1200, height=600, xrotation=90]
overlap_hv = hv.Dataset(overlaps, kdims=['session', 'group', 'overlap-type'], vdims=['overlap-value'])
overlap_hv.to(hv.BoxWhisker, kdims=['session', 'group'])

In [ ]:
%%opts Curve [width=1000, height=400, xrotation=90, tools=['hover']]
overlap_hv_anm = hv.Dataset(overlaps, kdims=['animal', 'session', 'group', 'overlap-type'], vdims=['overlap-value'])
overlap_hv_anm.to(hv.Curve, kdims=['session']).overlay('animal').layout('group').cols(1)